In [ ]:
import os
os.getcwd()

In [ ]:
import gc

In [ ]:
import pickle
import os
Name = ['MeltedToast', 'Mithraaaa']
feat = []
army = []
Y = []
id = []
for name in Name:
    file_path = '/content/Match_Data/'+name
    replays = os.listdir(file_path)
    for item in replays:
        replay_path = file_path + '/' + item
        with open(replay_path,"rb") as file:
            total = pickle.load(file)
            feat.append(total["features"])
            army.append(total["armies"])
            Y.append(total["kings"])
            id.append(total["id"])

# print(len(X))
# print(data[0])

In [ ]:
mx = 0
feat_new = []
army_new = []
Y_new = []
for item in range(0,10374):
    mx = max(mx, len(Y[item]))
    if (len(Y[item])<=1000):
        feat_new.append(feat[item])
        army_new.append(army[item])
        Y_new.append(Y[item])
        # print(id[item])
print(mx)
del feat, army, Y, id
gc.collect()
import random
random_integers = random.sample(range(0, 10001), 3000)
print(random_integers)
feat = [feat_new[item]for item in random_integers]
army = [army_new[item]for item in random_integers]
Y = [Y_new[item]for item in random_integers]

In [ ]:
del feat_new, army_new, Y_new
gc.collect()

In [ ]:
print(Y[0])

In [ ]:
import numpy as np
print(len(feat))
def transform_to_numpy(X):
    res = []
    for item in X:
        XX = []
        flag=0
        for item2 in item:
            try:
                # print(item2)
                # print("TRY1: ",get_list_depth(item2),len(item2),)
                XX.append(np.array(item2))
            except ValueError as e:
                print(item2)
                # print("TRY2: ",get_list_depth(item2),len(item2))
                flag=1
                break
        if flag:
            break
        res.append(np.array(XX))
    return res

feat_np = transform_to_numpy(feat)
army_np = transform_to_numpy(army)
Y_np = []
for item in Y:
    Y_np.append(np.array(item))
# print(Y_np[0])

In [ ]:
del feat, army, Y
gc.collect()

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
feat_flatten = [[step.flatten() for step in sequence] for sequence in feat_np]
feat_padded_flat = pad_sequences(feat_flatten, padding='post', value=-1.0)
feat_padded = np.array([np.array([step.reshape(25, 25) for step in sequence]) for sequence in feat_padded_flat])

army_flatten = [[step.flatten() for step in sequence] for sequence in army_np]
army_padded_flat = pad_sequences(army_flatten, padding='post', value=-1.0)
army_padded = np.array([np.array([step.reshape(25, 25) for step in sequence]) for sequence in army_padded_flat])

Y_flatten = [[step.flatten() for step in sequence] for sequence in Y_np]
Y_padded_flat = pad_sequences(Y_flatten, padding='post', value=-1.0)
Y_padded = np.array([np.array([step.reshape(1, 2) for step in sequence]) for sequence in Y_padded_flat])

In [ ]:
del feat_np,army_np,Y_np
gc.collect()

In [ ]:
print(feat_padded.shape)
print(army_padded.shape)
print(Y_padded.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, LSTM, Dense, TimeDistributed, Concatenate
from keras import backend as K

# model = Sequential()

input_1 = Input(shape=(None, 25, 25, 1))
input_2 = Input(shape=(None, 25, 25, 1))

conv1_1 = TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))(input_1)
conv1_1 = TimeDistributed(Flatten())(conv1_1)

conv1_2 = TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))(input_2)
conv1_2 = TimeDistributed(Flatten())(conv1_2)

merged = Concatenate(axis=-1)([conv1_1, conv1_2])
lstm_out = LSTM(units=64, return_sequences=True, recurrent_activation='sigmoid')(merged)

output = Dense(units=2, activation='linear')(lstm_out)

model = Model(inputs=[input_1, input_2], outputs=output)

def loss_func(y_true, y_pred):
    ignore_steps = 20
    y_true = tf.squeeze(y_true, axis=2)
    time_steps = tf.shape(y_true)[1]
    mask = tf.reduce_all(tf.equal(y_true, -1), axis=-1) 
    mask = tf.cast(mask, dtype=tf.float32)
    loss = tf.abs(y_pred[:, ignore_steps:, :] - y_true[:, ignore_steps:, :])
    loss = loss * (1 - mask[:, ignore_steps:])[:, :, None] 
    return tf.reduce_mean(loss)

model.compile(optimizer='adam', loss=loss_func)

model.summary()

In [ ]:
from sklearn.model_selection import train_test_split

X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(
    feat_padded, army_padded, Y_padded, test_size=0.2, random_state=42
)

In [ ]:
del feat_padded, army_padded, Y_padded
gc.collect()

In [ ]:
model.fit([X1_train, X2_train], y_train, epochs=10, batch_size=32, verbose=1)
model.save('model_full.h5')
model.evaluate([X1_test, X2_test], y_test, verbose=1)